In [1]:
import pandas as pd
import os
import numpy as np
import math
from skimpy import clean_columns

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [2]:
df = clean_columns(pd.read_csv(os.path.join(os.getcwd(),'validation_results_2020.csv')))
df.head()

6 column names have been cleaned

,plotid,center_lon,center_lat,shape,size_m,sample_points,email,flagged,flagged_reason,collection_time,...,common_securewatch_date,total_securewatch_dates,pl_class,pl_which_raster,category_low_ndvi_impervious_surface,category_non_iceplant_vegetation,category_iceplant,category_water,validation_finished_yes_high_confidence,validation_finished_no_low_confidence
0,0,-120.373228,34.454063,square,20.0,1,galaz-garcia@nceas.ucsb.edu,False,NaN,2022-11-05 00:30,...,NaN,0,3,0,0.0,0.0,0.0,100.0,100.0,0.0
1,1,-119.585374,34.424282,square,20.0,1,galaz-garcia@nceas.ucsb.edu,False,NaN,2022-11-05 00:31,...,NaN,0,1,2,0.0,100.0,0.0,0.0,100.0,0.0
2,2,-120.169518,34.471802,square,20.0,1,galaz-garcia@nceas.ucsb.edu,False,NaN,2022-11-08 20:05,...,NaN,0,0,0,0.0,100.0,0.0,0.0,100.0,0.0
3,3,-120.008978,34.466743,square,20.0,1,galaz-garcia@nceas.ucsb.edu,False,NaN,2022-11-08 18:00,...,NaN,0,1,0,0.0,100.0,0.0,0.0,100.0,0.0
4,4,-119.977029,34.457047,square,20.0,1,galaz-garcia@nceas.ucsb.edu,False,NaN,2022-11-08 18:03,...,NaN,0,0,2,0.0,100.0,0.0,0.0,0.0,100.0


In [3]:
def ref_class_column(df):

    map_class = df.pl_class
    ref_class = []

    for i in map_class.index:
        if df.category_non_iceplant_vegetation.loc[i] == 100:
            ref_class.append(0)
        elif df.category_iceplant.loc[i] == 100:
            ref_class.append(1)
        elif df.category_low_ndvi_impervious_surface.loc[i] == 100:
            ref_class.append(2)
        elif df.category_water.loc[i] == 100:
            ref_class.append(3)
        else:
            ref_class[j]= 100
            
    return ref_class

In [4]:
df['ref_class'] = ref_class_column(df)
df = df.drop(['center_lon', 'center_lat', 'shape', 'size_m', 'sample_points',
        'flagged', 'flagged_reason', 'collection_time', 
         'total_securewatch_dates', 'common_securewatch_date',
        'pl_which_raster', 
         'validation_finished_yes_high_confidence',
         'analysis_duration',
         'category_low_ndvi_impervious_surface','category_non_iceplant_vegetation',
         'category_iceplant', 'category_water',], axis =1)
df = df.rename( columns = {'pl_class':'map_class',
                       'validation_finished_no_low_confidence':'low_confidence'})

In [5]:
np.unique(df.ref_class, return_counts=True)

(array([0, 1, 2, 3]), array([136,  53, 143, 133]))

In [6]:
np.unique(df.map_class, return_counts=True)

(array([0, 1, 2, 3]), array([100, 100, 150, 115]))

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# using confusion_matrix directly we get a matrix C such that
# C_{i,j} = known to be i, predicted as  j 
# The notation in the paper is 
# n_{i,j} = predicted as i, known to be j 
# so we need to take the transpose

n = confusion_matrix(df.ref_class, df.map_class, labels=range(0,4)).T
n

array([[ 79,   2,  14,   5],
       [ 37,  49,   9,   5],
       [ 20,   2, 117,  11],
       [  0,   0,   3, 112]])

In [10]:
pix_counts = pd.read_csv(os.path.join(os.getcwd(), 'rasters_2020_pixel_counts.csv'))
total_pix = sum([sum(pix_counts.n_nonice_20),
                  sum(pix_counts.n_ice_20),
                  sum(pix_counts.n_ground_20),
                  sum(pix_counts.n_water_20)])

W = []      # proportion of area mapped as class i
n_idot = [] # pixels in sample that had class i in map (predicted as i, any true class j)
U_hat = []  # estimated users' accuracy (precision for each class: TP/(TP+FP))
#P_hat = []  # estimated producer's accurace (sensitiviy for each class TP/(TP+FN))

for i in range(0,4):
    W.append( sum(pix_counts.iloc[:,i]) / total_pix)
    n_idot.append(sum(n[i,:]))
    U_hat.append(n[i,i] / n_idot[i])
    

In [21]:
O = sum([W[i]*n[i,i]/n_idot[i] for i in range(0,4)])
print('overall accuracy:', O*100)

var_O = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)])
# std error of estimated overall accuracy -- paper equation (5)
print('overall accuracy std error:', np.sqrt(var_O)*100, '\n')

print('users accuracy:', U_hat)

var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)]
print('users accuracies std errors:', np.sqrt(var_U_hat)*100)

overall accuracy: 83.34944453527964
overall accuracy std error: 1.8087344195503947 

users accuracy: [0.79, 0.49, 0.78, 0.9739130434782609]
users accuracies std errors: [4.09360181 5.02418394 3.39363795 1.492861  ]


In [15]:
high_confidence = df[df.low_confidence == 0]
high_confidence

,plotid,email,map_class,low_confidence,ref_class
0,0,galaz-garcia@nceas.ucsb.edu,3,0.0,3
1,1,galaz-garcia@nceas.ucsb.edu,1,0.0,0
2,2,galaz-garcia@nceas.ucsb.edu,0,0.0,0
3,3,galaz-garcia@nceas.ucsb.edu,1,0.0,0
5,5,galaz-garcia@nceas.ucsb.edu,2,0.0,0
...,...,...,...,...,...
460,460,galaz-garcia@nceas.ucsb.edu,0,0.0,0
461,461,galaz-garcia@nceas.ucsb.edu,2,0.0,2
462,462,galaz-garcia@nceas.ucsb.edu,0,0.0,0
463,463,galaz-garcia@nceas.ucsb.edu,2,0.0,0


In [22]:
n = confusion_matrix(high_confidence.ref_class, high_confidence.map_class, labels=range(0,4)).T
n

array([[ 73,   1,   7,   5],
       [ 21,  42,   7,   5],
       [ 14,   0, 107,  10],
       [  0,   0,   3, 110]])

In [26]:
n_idot = [] # pixels in sample that had class i in map (predicted as i, any true class j)
U_hat = []  # estimated users' accuracy (precision for each class: TP/(TP+FP))

for i in range(0,4):
    n_idot.append(sum(n[i,:]))
    U_hat.append(n[i,i] / n_idot[i])

O = sum([W[i]*n[i,i]/n_idot[i] for i in range(0,4)])
print('overall accuracy:', O*100)

var_O = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)])
# std error of estimated overall accuracy -- paper equation (5)
print('overall accuracy std error:', np.sqrt(var_O)*100, '\n')

print('users accuracy:', U_hat)

var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1)for i in range(0,4)]
var_U_hat
print('users accuracies std errors:', np.sqrt(var_U_hat)*100)

overall accuracy: 86.59108061798332
overall accuracy std error: 1.7778445287918678 

users accuracy: [0.8488372093023255, 0.56, 0.816793893129771, 0.9734513274336283]
users accuracies std errors: [3.88530567 5.77038104 3.39277093 1.51904107]


In [27]:
196*np.sqrt(var_U_hat)

array([ 7.61519912, 11.30994683,  6.64983102,  2.97732049])

In [28]:
np.sqrt(var_O)*196

3.484575276432061